In [11]:
import pandas as pd
import numpy as np 
import torch
import torch.nn as nn
import random
import torch.nn.functional as F

In [12]:
BATCH_SIZE = 32
LR = 0.01
MEMORY_LEN = 3000
GAMMA = 0.9
EPSILON = 0.9
EPOCH = 100
UPDATE_FREQUENCY = 64

In [13]:
stock_path = "/Users/ted/Desktop/lab/2330Inf.csv"
test_path = "/Users/ted/Desktop/lab/3034Inf.csv"
stock_df = pd.read_csv(stock_path)
test_df = pd.read_csv(test_path)
stock_df2 = pd.DataFrame(stock_df,columns=["Open","High","Low","Close","K value","D value","EMA 12","DIF","MACD"])
test_df2 = pd.DataFrame(stock_df,columns=["Open","High","Low","Close","K value","D value","EMA 12","DIF","MACD"])
# print(stock_df2)

# declear training features data
features = []
for i in range(1,len(stock_df2)):
    x = stock_df2[i-1:i][["Open","High","Low","Close","K value","D value","EMA 12","DIF","MACD"]].values
    features.append(x.tolist())
features = np.array(features)
# print("size of feature: ",features.size())
# print(features[0].shape)


# declear trainging labels data
test_features = []
for i in range(1,len(test_df2)):
    x = stock_df2[i-1:i][["Open","High","Low","Close","K value","D value","EMA 12","DIF","MACD"]].values
    test_features.append(x.tolist())
test_features = np.array(test_features)
#print("size of feature: ",test_features.size())
test_features

array([[[ 59.799999  ,  60.400002  ,  59.299999  , ...,  59.44179856,
          -0.69113272,  -0.87297582]],

       [[ 60.799999  ,  61.        ,  60.5       , ...,  59.65075247,
          -0.53159123,  -0.8046989 ]],

       [[ 60.5       ,  61.        ,  60.200001  , ...,  59.85832902,
          -0.38458182,  -0.72067549]],

       ...,

       [[294.        , 294.        , 291.5       , ..., 294.79525067,
           0.79173718,   1.7364945 ]],

       [[293.        , 294.        , 291.        , ..., 294.67290441,
           0.66965118,   1.52312584]],

       [[296.        , 298.        , 295.        , ..., 295.10784219,
           0.84557069,   1.38761481]]])

In [14]:
class Environment:
    #inital environment 
    def __init__(self,env1):
        self.day = 0             
        self.env1 = env1         # size:  (2254 , 30 , 6)
        self.stockNum = 0        # num of stock
        self.money = 10000       # initial budget
        
    # reset environment  , return first state
    def reset(self):
        self.day = 0             
        self.stockNum = 0       
        self.money = 10000
        return self.env1[0]      # first state
    
    # return n-day's state
    def get_observation(self):
        return self.env1[self.day] 
    
    #return action space [ sell, hold , buy] 
    def get_actions(self,n):
        a = ["sell","hold","buy"]
        return a[n] # [sell,hold,buy]
    
    # if finish or not
    def is_done(self):
        return self.day == 2445
    
    
    # input : action  output :　reward    
    def actions(self,action,date):
        self.day = date
        reward = 0
        if(action == 0):     # sell
            self.money += (self.stockNum*self.env1[self.day][0][3])   #  sell all stock
            self.stockNum = 0
            reward = self.env1[self.day][0][3] - self.env1[self.day+1][0][3]   #calculate reward
        elif(action == 1):   #hold
            reward =  -1* self.env1[self.day+1][0][3]
        elif(action == 2):   #buy
            num = self.money // self.env1[self.day][0][3]
            self.stockNum += num                             # buy as many as stock as agent can
            self.money -= (num * self.env1[self.day][0][3])
            reward = self.env1[self.day+1][0][3] - self.env1[self.day][0][3]           #calculate reward
        self.day += 1          # next day
        return reward
        
        
#     def step(self,act_probs):
        
#         act_probs = act_probs.data.numpy()[0]                    # get the probability of each action (sell , hold ,buy)
#         action = np.random.choice(len(act_probs), p=act_probs)   # random choose action but the action with higher probability 
#                                                                  # would have higher chance to be choose
#         if(action == 0 ):  #sell                                   
#             if(self.stockNum == 0):                              # if agent don't have any stock to sell
#                 if(self.money >= self.close[self.day]):          # if agent have enough budget for buying stocks 
#                     space = [1,2]                   
#                     action = np.random.choice(space)             # random choose buy or hold
#                 else:
#                     action = 1                                   # otherwise choose hold
                    
#         elif(action == 2): # buy
#             if(self.money < self.close[self.day]):               # if agent don't have enough meney for buying stocks
#                 if(self.stockNum != 0):                          # if agent have one or more stock to sell
#                     space = [0,1]
#                     action = np.random.choice(space)             # random choose sell or hold
#                 else:
#                     action = 1                                   # otherwise choose hold
                    
#         #print("Day %d , Action: %s"%(self.day ,self.get_actions(action)),end = ' ')
#         reward = self.actions(action)                            # get reward
#         #print("Reward: %.4f , StockNum: %d ,Budget: %.4f"%(reward,self.stockNum,self.money))
        
#         if(self.is_done()):                                      # if finish , print the rate of return
#             print("---------------------------------------------")
#             print("Total Reward: %lf" % (((self.stockNum * self.close[self.day]) + self.money)*100/10000))
#             print("---------------------------------------------")
#         return self.get_observation(), reward, self.is_done(),action

In [15]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 32)
        self.fc1.weight.data.normal_(0, 0.1)   # initialization
        self.fc2 = nn.Linear(32,64)
        self.fc2.weight.data.normal_(0, 0.1)
        self.out = nn.Linear(64, 3)
        self.out.weight.data.normal_(0, 0.1)   # initialization
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value

In [16]:
class DDQN():
    def __init__(self):
        self.eval_net,self.target_net = Net(),Net()
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)    
#         self.loss_func = nn.MSELoss()
        self.memory_counter = 0
        self.memory = np.zeros((MEMORY_LEN, 20))
#         self.memory = np.zeros((MEMORY_LEN,MEMORY_LEN))
        
        
    def store_memory(self,s,a,r,s_):
        index = self.memory_counter % MEMORY_LEN
        transition = np.hstack((s, [a, r], s_))
        self.memory[index,:] = transition
        self.memory_counter += 1
    
    def train(self,loss):
        self.optimaizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    
    def learn(): #更新兩個network的參數
#         sample_index = np.random.choice(MEMORY_LEN,BATCH_SIZE)
#         b_memory = self.memory[sample_index,:]
#         b_r = torch.FloatTensor(b_memory[])
        self.target_net.load_state_dict(self.eval_net.state_dict())
        
    def choose_action(self,x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        if np.random.uniform() <= EPSILON:   # greedy
            target_value = self.eval_net.forward(x)
            if(target_value[0][0]>target_value[0][1] and target_value[0][0]>target_value[0][2]):
                return 0
            if(target_value[0][1]>target_value[0][2] and target_value[0][1]>target_value[0][0]):
                return 1
            if(target_value[0][2]>target_value[0][1] and target_value[0][2]>target_value[0][0]):
                return 2
#             action = torch.max(actions_value, 1)    # return the argmax
        else:   # non-greedy
            action = np.random.randint(1,size=(0,2))
        return action
    def eval_Qtarget(self,x):
        x = torch.unsqueeze(torch.FloatTensor(x),0)
        target_value = self.target_net(x)
#         print(target_value[0])
#         if(target_value[0]>target_value[1] and target_value[0]>target_value[2]):
#             return target_value[0]
#         if(target_value[1]>target_value[2] and target_value[1]>target_value[0]):
#             return target_value[1]
#         if(target_value[2]>target_value[1] and target_value[2]>target_value[0]):
#             return target_value[2]
#         target_value_ = torch.max(target_value,1)
#         target = target_value_.item()
#         target_value = torch.FloatTensor(target_value_)
        return target_value
        

In [17]:
from collections import namedtuple
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [18]:
def iterate_batch(env,net,batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    
    obs = env.reset()        # initialize enveriment
    
    sm = nn.Softmax(dim = 1)
    
    while True:
        obs_v = torch.FloatTensor(obs)      # numpy to tensor
        obs_v = obs_v.view(-1,30,6)         # reshape to (batch_size, time_steps , input_size)
        out = net(obs_v)
        act_probs = sm(out)                 # calculate the probability of each action
    
        next_obs , reward , is_done,action = env.step(act_probs)  # do the action , return (next state , reward , is_done , action)
        
        episode_reward += reward            # calculate total of reward
        episode_steps.append(EpisodeStep(observation = obs,action = action))
        
        if is_done:                         # if finish one round
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()          # initialize environment
            if len(batch) == batch_size:    # if finish all rounds
                yield batch
                batch = []
        obs = next_obs

In [19]:
ddqn = DDQN()

In [ ]:
env = Environment(features)
obs_size = 9
n_actions = 3
for i in range(EPOCH):
    #initialize sequence s1 = {x1} and preprocessed sequenced
    for j in range(len(features)-1):
        state = features[j][0]
        state_ = features[j+1][0]
        Q_value = ddqn.choose_action(state)
        Q_value_next = ddqn.choose_action(state_)
        Q_value_ = ddqn.eval_Qtarget(state_)
#         print(Q_value_)
# #         Q_value_ = Q_value_.item()
# #         Q_value_next = float(Q_value_next)
# #         Q_target = Q_value_next + GAMMA * Q_value_
#         reward = env.actions(Q_value,j)
#         # action,reward,state,next_state
# #         ddqn.store_memory(state,Q_value,reward,state_)
#         if(j%UPDATE_FREQUENCY==0):
#             pass
# #           ddqn.learn()
# #         print(action)
# #         loss = loss_func(Q_value,Q_target)
# #         train(loss)